# Distributed DeepRacer RL training with SageMaker and RoboMaker

---
## Introduction


In this notebook, we will train a fully autonomous 1/18th scale race car using reinforcement learning using Amazon SageMaker RL and AWS RoboMaker's 3D driving simulator. [AWS RoboMaker](https://console.aws.amazon.com/robomaker/home#welcome) is a service that makes it easy for developers to develop, test, and deploy robotics applications.  

This notebook provides a jailbreak experience of [AWS DeepRacer](https://console.aws.amazon.com/deepracer/home#welcome), giving us more control over the training/simulation process and RL algorithm tuning.

![Training in Action](./deepracer-reinvent-track.jpg)


---
## How it works?  

![How training works](./training.png)

The reinforcement learning agent (i.e. our autonomous car) learns to drive by interacting with its environment, e.g., the track, by taking an action in a given state to maximize the expected reward. The agent learns the optimal plan of actions in training by trial-and-error through repeated episodes.  
  
The figure above shows an example of distributed RL training across SageMaker and two RoboMaker simulation envrionments that perform the **rollouts** - execute a fixed number of episodes using the current model or policy. The rollouts collect agent experiences (state-transition tuples) and share this data with SageMaker for training. SageMaker updates the model policy which is then used to execute the next sequence of rollouts. This training loop continues until the model converges, i.e. the car learns to drive and stops going off-track. More formally, we can define the problem in terms of the following:  

1. **Objective**: Learn to drive autonomously by staying close to the center of the track.
2. **Environment**: A 3D driving simulator hosted on AWS RoboMaker.
3. **State**: The driving POV image captured by the car's head camera, as shown in the illustration above.
4. **Action**: Six discrete steering wheel positions at different angles (configurable)
5. **Reward**: Positive reward for staying close to the center line; High penalty for going off-track. This is configurable and can be made more complex (for e.g. steering penalty can be added).

### Imports

To get started, we'll import the Python libraries we need, set up the environment with a few prerequisites for permissions and configurations.

You can run this notebook from your local machine or from a SageMaker notebook instance. In both of these scenarios, you can run the following to launch a training job on SageMaker and a simulation job on RoboMaker.

In [1]:
from src.core.DeepRacerEngine import DeepRacerEngine

### Scenario 1: Single Model Run and Evaluation

For this scenario, we are going to run a single model, over a chosen track

### Set Up Parameters
There are two set of parameters we can configure:
 - The simulation parameters
 - The model hyperparameters

#### View Default Simulation Parameters

In [2]:
!pygmentize common/constant.py

# Estimator Pamrs
entry_point = "training_worker.py"
source_dir = 'src'

#Training Params
default_instance_type = "ml.c4.2xlarge" #For GPU use 'ml.p3.2xlarge'
default_instance_pool = 1
default_job_duration = 3600
default_hyperparam_preset = 'src/markov/presets/preset_hyperparams.json'
tmp_hyperparam_preset = 'src/markov/presets/preset_hyperparams_tmp.py'

#Track Details:
default_track_name = 'reinvent_base'
track_name = ['reinvent_base', 'reinvent_carpet', 'reinvent_concrete', 'reinvent_wood', 'AWS_track',
              'Bowtie_track', 'Oval_track',  'Straight_track']

# Evaluation Trials
evaluation_trials = 5

## Policy and Model Meta Data
envir_file_local = 'src/markov/environments/deepracer_racetrack_env.py'
reward_file_local = 'src/markov/rewards/complex_reward.py'
model_meta_file_local = 'src/markov/actions/model_metadata_10_state.json'
presets_file_local = 'src/markov/presets/preset_attention_layer.py'


#### View Default Hypterparameters

In [3]:
!pygmentize src/markov/presets/preset_hyperparams.json

{
    "learning_rate": 0.0003,
    "batch_size" : 64,
    "optimizer_epsilon" : 0.00001,
    "adam_optimizer_beta2" : 0.999,
    "clip_likelihood_ratio_using_epsilon" : 0.2,
    "beta_entropy" : 0.01,
    "gae_lambda" : 0.95,
    "discount" : 0.999,
    "optimization_epochs" : 10
}


#### Set Parameters

In [3]:
#here we can set the params we want to use for our model runs
params = {
    'job_name': 'dr-test-abc',
    'track_name':'reinvent_base',
    'job_duration': 3600,
    'batch_size':128,
    'evaluation_trials':5
}

#### Instantiate the DeepRacerEngine class, and provide the params

In [6]:
deepRacer = DeepRacerEngine(params)

***Deep Racer Engine Backend***


#### Start Simulation Training 

In [ ]:
deepRacer.start_training_testing_process()

#### Plot Training Process

In [ ]:
deepRacer.plot_training_output()

Create local folder tmp/dr-test-abc-2019-12-24-05-51-39-591


#### Start Evaluation Proces


In [ ]:
deepRacer.start_evaluation_process()

#### Plot Evaluation Process

In [ ]:
deepRacer.plot_evaluation_output()

====================

### Scenario 2: Multi-Model Training with different HyperParameters and Evaluation

For this scenario, we are going to train multiple models, with different hyperparameters, and view the training and evaluation in parallel.

#### Hyperparameter Generation

In this example we're going to generate some changes to the hyperparameters. Typically when conducting experiments with varible chances, only one variable is adjusted at a time, in order to allow for measurable changes (controlled experiment). If multiple variables are changed, then it is difficult to determine the impact to the hypothesis.

In [ ]:
def param_gen_batch_sizes(self, min_batch = 64, max_batch = 512, job_name_prefix):
    
    batches = []
    btch = min_batch 
    while btch <= max_batch:
        batches.append(btch)
        btch *= 2
    print(batches)
    
    model_params = []
    job_name = job_name_prefix+'-batchsize-'
    for batch_size in batches:
        
        params = {
        'job_name': job_name+'{}'.format(batch_size),
        'track_name':'reinvent_base',
        'job_duration': 3600,
        'batch_size':batch_size,
        'evaluation_trials':5
        }
    model_params.append(params)
    
param_gen_batch_sizes(None)